In [ ]:
!pip install pytorch_lightning

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl


import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_test_flat = x_test.reshape(x_test.shape[0], 784)

x_train_tensor = torch.tensor(x_train.reshape(x_train.shape[0], 784)).float()
y_train_tensor = torch.tensor(y_train).long() # CrossEntropyLoss expects Long type

x_test_tensor = torch.tensor(x_test.reshape(x_test.shape[0], 784)).float()
y_test_tensor = torch.tensor(y_test).long() # CrossEntropyLoss expects Long type

In [ ]:
print(y_train_tensor[0])
plt.imshow(x_train[0], cmap='gray')
plt.show()

In [12]:
class LitSimpleNN(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # logits
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log("test_loss", loss)
        self.log("test_acc", acc)
model = LitSimpleNN()

In [13]:
from torch.utils.data import TensorDataset, DataLoader

# Suppose x_train_tensor is (N, 784) and y_train_tensor is (N,)
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
from pytorch_lightning import Trainer

model = LitSimpleNN()
trainer = Trainer(max_epochs=1)
trainer.fit(model, train_loader)

In [ ]:
trainer.test(model, test_loader)